# Q4 — New System Design & Evaluation (主线)

目的：在可解释、可复现的前提下，对候选新赛制（投票机制）做“多指标评估”：
- **TPI**：技术保护指数（越高越不容易让技术差的纯人气选手夺冠）
- **Fan expression**：粉丝表达强度（这里用 fan_vs_uniform_contrast 做受控指标）
- **Robustness**：对粉丝离群值（极端动员/黑天鹅）的稳健性

对应主线代码：
- `src/mcm2026/pipelines/mcm2026c_q4_design_space_eval.py`
- `src/mcm2026/visualizations/q4_visualizations.py`

主要输出（表）：
- `outputs/tables/mcm2026c_q4_new_system_metrics.csv`（固定文件名，保证兼容）
- 以及一份 **参数戳** 的副本（文件名包含 fan_source_mechanism / n_sims / seed / outlier_mults / bootstrap_b / sigma_scales 等）

本 notebook 特别说明两块：
1) **误差分析 / 不确定性量化**：主表里有哪些 `*_se`、CI 列，怎么解释
2) **灵敏度检测**：如何通过 `sigma_scales`（以及 n_sims）做稳健性论证


In [ ]:
from __future__ import annotations

import sys
from pathlib import Path

import numpy as np
import pandas as pd
import yaml

def _find_repo_root(start: Path | None = None) -> Path:
    here = Path.cwd() if start is None else Path(start)
    for p in [here, *here.parents]:
        if (p / 'pyproject.toml').exists():
            return p
    return here

REPO_ROOT = _find_repo_root()
SRC_DIR = REPO_ROOT / 'src'
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

REPO_ROOT


In [ ]:
# 工具函数：在 notebook 里自动展示 outputs/figures 下的图片（优先 TIFF，其次提示 EPS 路径）
from pathlib import Path


def show_saved_figures(question: str, stems: list[str]) -> None:
    """Display figures saved by visualization scripts.

    - Prefer TIFF (good for notebook preview)
    - EPS usually cannot be previewed in Jupyter without额外系统依赖，所以这里只打印路径
    """

    try:
        from IPython.display import Image, display
    except Exception:
        print('⚠️ IPython not available; cannot display images inline.')
        return

    q = question.strip().lower()
    tiff_dir = Path(REPO_ROOT) / 'outputs' / 'figures' / q / 'tiff'
    eps_dir = Path(REPO_ROOT) / 'outputs' / 'figures' / q / 'eps'

    shown = 0
    for stem in stems:
        tiff_fp = tiff_dir / f'{stem}.tiff'
        eps_fp = eps_dir / f'{stem}.eps'

        if tiff_fp.exists():
            display(Image(filename=str(tiff_fp)))
            shown += 1
        elif eps_fp.exists():
            print('EPS figure exists (not previewed inline):', eps_fp)

    if shown == 0:
        print('⚠️ No TIFF figures found under:', tiff_dir)


# 用法示例（Q4）：
# show_saved_figures('q4', ['q4_mechanism_tradeoff_scatter', 'q4_robustness_curves'])

## 一键运行（推荐给新队友/论文手）

Q4 是最适合“一键跑完 + 直接看图 + 直接看误差分析/灵敏度”的部分。

运行下面这一格会：
- 自动跑 Q0 → Q1 → Q4（补齐依赖）
- 读取 `config.yaml` 并打印关键配置（n_sims / bootstrap_b / sigma_scales / outlier_mults）
- 生成 Q4 图（TIFF/EPS）
- 在 notebook 里直接展示关键 TIFF 图

如果中途报错，会尽量给“像人一样的提示”，告诉你下一步该检查什么。

In [ ]:
import traceback

try:
    import sys
    from pathlib import Path

    import pandas as pd

    # ---- Make this cell self-contained (no dependency on earlier cells) ----
    def _find_repo_root(start: Path | None = None) -> Path:
        here = Path.cwd() if start is None else Path(start)
        for p in [here, *here.parents]:
            if (p / 'pyproject.toml').exists():
                return p
        return here

    if 'REPO_ROOT' not in globals() or not isinstance(globals().get('REPO_ROOT'), Path):
        REPO_ROOT = _find_repo_root()

    SRC_DIR = REPO_ROOT / 'src'
    if str(SRC_DIR) not in sys.path:
        sys.path.insert(0, str(SRC_DIR))

    try:
        from IPython.display import display
    except Exception:
        def display(x):
            return x

    from mcm2026.pipelines import (
        mcm2026c_q0_build_weekly_panel,
        mcm2026c_q1_smc_fan_vote,
        mcm2026c_q4_design_space_eval,
    )
    from mcm2026.visualizations.config import VisualizationConfig, create_output_directories
    from mcm2026.visualizations.q4_visualizations import generate_all_q4_visualizations

    _ = mcm2026c_q0_build_weekly_panel.run()
    _ = mcm2026c_q1_smc_fan_vote.run()

    q4_out = mcm2026c_q4_design_space_eval.run()
    print('✅ Q4 done:', q4_out)

    metrics_fp = REPO_ROOT / 'outputs' / 'tables' / 'mcm2026c_q4_new_system_metrics.csv'
    m = pd.read_csv(metrics_fp)
    print('rows:', len(m), 'cols:', len(m.columns))

    key_cols = [
        'season', 'mechanism', 'outlier_mult', 'n_sims', 'sigma_scale',
        'tpi_season_avg', 'tpi_boot_p025', 'tpi_boot_p975',
        'fan_vs_uniform_contrast', 'fan_vs_uniform_contrast_se',
        'robust_fail_rate', 'robust_fail_rate_se',
    ]
    key_cols = [c for c in key_cols if c in m.columns]
    display(m[key_cols].head(8))

    viz_cfg = VisualizationConfig()
    out_dirs = create_output_directories(REPO_ROOT / 'outputs' / 'figures', ['Q4'])
    generate_all_q4_visualizations(REPO_ROOT, out_dirs['Q4'], viz_cfg, showcase=False)

    # Show TIFFs (best-effort)
    try:
        from IPython.display import Image, display as _display

        tiff_dir = REPO_ROOT / 'outputs' / 'figures' / 'q4' / 'tiff'
        candidates = [
            tiff_dir / 'q4_mechanism_tradeoff_scatter.tiff',
            tiff_dir / 'q4_robustness_curves.tiff',
            tiff_dir / 'q4_champion_uncertainty_analysis.tiff',
            tiff_dir / 'q4_mechanism_recommendation.tiff',
        ]

        shown = 0
        for fp in candidates:
            if fp.exists():
                _display(Image(filename=str(fp)))
                shown += 1
        if shown == 0:
            print('⚠️ No TIFF figures found to display yet. They should be under:', tiff_dir)
    except Exception:
        pass

except Exception as e:
    print('❌ One-click Q4 failed:', e)
    traceback.print_exc()

## 1) 查看当前配置（重点：误差分析与灵敏度）

Q4 的关键配置在 `src/mcm2026/config/config.yaml` 的 `dwts.q4` 节点。

- `n_sims` 越大：Monte Carlo 标准误越小（但更慢）
- `bootstrap_b`：TPI 的 bootstrap CI 采样次数（越大越稳但更慢）
- `sigma_scales`：用来做 **Q1→Q4 不确定性传播强度** 的灵敏度分析（默认只跑 1.0）
- `robustness_attacks.enabled`：扩展鲁棒性攻击（默认关闭，避免主线变慢）

In [ ]:
cfg_fp = REPO_ROOT / 'src' / 'mcm2026' / 'config' / 'config.yaml'
cfg = yaml.safe_load(cfg_fp.read_text(encoding='utf-8'))
dwts_q4 = (cfg or {}).get('dwts', {}).get('q4', {})
dwts_q4


## 2) 运行 Q4

Q4 会对每个 season、每个 mechanism、每个 outlier_mult（压力测试强度），做 `n_sims` 次赛季模拟，并汇总为主表一行。

说明：如果你只想快速验证流程，把 `n_sims` 改小（你现在是 20）；如果要写论文、让误差条更可信，建议加大到 50/100 甚至更高。

In [ ]:
from mcm2026.pipelines import mcm2026c_q0_build_weekly_panel, mcm2026c_q1_smc_fan_vote
from mcm2026.pipelines import mcm2026c_q4_design_space_eval

_ = mcm2026c_q0_build_weekly_panel.run()
_ = mcm2026c_q1_smc_fan_vote.run()

q4_out = mcm2026c_q4_design_space_eval.run()
q4_out


## 3) 检查：主表是否包含误差分析列（SE/CI）

你当前生成的主表已经包含：
- `champion_mode_prob_se`：冠军模式概率的 Monte Carlo SE
- `fan_vs_uniform_contrast_se`：同上（把 fan 指标当作均值估计）
- `robust_fail_rate_se`：稳健性失败率的 SE（基于二项近似：p(1-p)/n）
- `tpi_std` / `tpi_p05` / `tpi_p95`：TPI 的分布信息
- `tpi_boot_p025` / `tpi_boot_p975`：TPI 的 bootstrap 95% CI（如果 bootstrap_b>0）

另外还预留了扩展攻击列（默认是空）：
- `robust_fail_rate_fixed` / `robust_fail_rate_random_bottom_k` / `robust_fail_rate_add` / `robust_fail_rate_redistribute`
（启用 `robustness_attacks.enabled: true` 后才会写入）

In [ ]:
metrics_fp = REPO_ROOT / 'outputs' / 'tables' / 'mcm2026c_q4_new_system_metrics.csv'
m = pd.read_csv(metrics_fp)
m.columns.tolist()


In [ ]:
m[['n_sims','sigma_scale','tpi_season_avg','tpi_boot_p025','tpi_boot_p975','fan_vs_uniform_contrast','fan_vs_uniform_contrast_se','robust_fail_rate','robust_fail_rate_se']].head(8)


## 4) 口语化解释：这张 Q4 主表怎么读？

你可以把 `mcm2026c_q4_new_system_metrics.csv` 理解成一张“机制体检表”。
每一行回答的是：
> 在某个季度（season），用某个机制（mechanism），在某个压力测试强度（outlier_mult）下，
> 我们模拟了 `n_sims` 次，得到这套机制的多指标表现。

### 一行里最关键的三类指标
1) **技术保护（TPI）**：`tpi_season_avg`
   - 同时给了 `tpi_boot_p025/p975`（bootstrap 95% CI）和 `tpi_std/p05/p95`
2) **粉丝表达（Fan expression）**：`fan_vs_uniform_contrast`
   - 给了 `fan_vs_uniform_contrast_se`（Monte Carlo 标准误）
3) **稳健性（Robustness）**：`robust_fail_rate`
   - 给了 `robust_fail_rate_se`（二项近似标准误）

### “误差分析”在这里到底是什么意思？
一句话：
> 我们不是只给一个点，而是告诉你这个点有多不稳（误差条有多长）。

### “灵敏度检测”在这里怎么做？
看 `sigma_scale`：
- `sigma_scale=1.0` 是默认口径
- 把 `sigma_scales` 改成 `[0.5, 1.0, 2.0]` 重跑后，你会得到同一机制在不同不确定性强度下的表现

下面我们直接用主表在 notebook 里画两张最直观的图：
- **Tradeoff scatter**：每个机制在“技术保护 vs 粉丝表达”的位置（带误差条）
- **Robustness curve**：压力测试从 2x→5x→10x 时失败率怎么变化（带误差带）

## 4) 灵敏度检测：sigma_scales

当前数据里 `sigma_scale` 已经写入主表（你目前配置是 `[1.0]`，所以只有 1 个取值）。

如果要做灵敏度分析：
1) 在 `config.yaml` 里把 `dwts.q4.sigma_scales` 改为例如 `[0.5, 1.0, 2.0]`
2) 重新运行 Q4
3) 主表会出现多组 `sigma_scale`（其它维度不变），可以比较指标是否“结论不变”。

注意：sigma_scales 是对 Q1→Q4 传播不确定性的强度缩放（越大代表 Q1 更不确定）。

In [ ]:
m['sigma_scale'].value_counts(dropna=False)


## 常见问答（FAQ）

### 为什么我们要做 bootstrap CI？为什么不是“一个 std 就完事”？
直觉解释：
- `tpi_season_avg` 是一个“模拟出来的均值”，均值本身也会波动
- 只看 std 你不知道“均值的置信范围”有多大

bootstrap CI 给的是：
> 如果我们把这 `n_sims` 次模拟当成样本，均值大概会落在哪个范围（95%）

对论文手来说，这比只给点估计更容易写出“科学严谨”的论证。

### 为什么 Q4 不用 p-value？
Q4 是 **模拟评估**：我们在比较机制指标，不是做“某个参数是否显著不为 0”的假设检验。
更合适的表达是：
- 点估计 + 误差条（SE/CI）
- 以及在不同压力条件/不确定性条件下是否稳定（鲁棒性/灵敏度）

### `sigma_scales` 是什么？为什么叫灵敏度？
Q1 给了 `fan_share_mean/p05/p95`。
Q4 在模拟时会从这些区间构造一个分布去抽样。

`sigma_scale` 做的事就是：
- 把这个抽样分布的“波动幅度”整体放大/缩小

所以它回答的问题是：
> 如果 Q1 更不确定（或更确定），我们的 Q4 结论是否还站得住？

### `n_sims` 该取多少？
经验法则（口语版）：
- 你想要误差条更细，就加大 `n_sims`
- 二项类指标的 SE 大约是 `sqrt(p(1-p)/n_sims)`，所以 n 从 20 提到 80，误差会大概减半。

建议：
- 调试：20
- 正式出图：50~100（时间允许的话更高）

### 扩展鲁棒性攻击（fixed/random_bottom_k/add/redistribute）什么时候开？
默认关闭是为了主线速度。

写论文/答辩时，如果你想强调“我们做了系统性的鲁棒性攻击测试”，可以打开：
- `dwts.q4.robustness_attacks.enabled: true`

注意：开了会变慢（因为每行要多跑一些攻击情景）。

## 6) 主线敏感性汇总表（跨季度聚合）怎么用？

为了让“灵敏度/稳健性分析”更像论文中的一段完整实验，我们在主线额外输出了一张汇总表：
- `outputs/tables/mcm2026c_q4_sensitivity_summary.csv`

你可以把它理解成：
> 把主表按 (mechanism, alpha, sigma_scale, outlier_mult) 分组，
> 然后跨 seasons 做聚合，输出“跨季度平均表现 + 区间 + 组合SE/CI”。

### 为什么这张表对论文手特别有用？
因为论文通常写的是“整体结论”，不是逐季列 34 个 season。
这张表可以直接支持：
- 我们推荐的机制在总体上是否更稳健
- 在不同 `sigma_scale`（Q1不确定性强弱）下结论是否保持
- 在不同 outlier_mult（压力强度）下稳健性曲线是否保持

### 这张表里有哪些关键列？
- `tpi_mean / fan_mean / robust_fail_mean`：跨 season 的平均表现
- `*_q05 / *_q95`：跨 season 的分位数范围（描述跨季度差异）
- `*_se / *_ci95_low/high`：我们组合了“跨季度波动 + 单季 Monte Carlo 误差”的一个近似 95% 区间
- `robust_fail_worst_mean`：如果启用扩展攻击，会额外提供 worst-case 口径的失败率均值

下面一格会直接读入这张表，展示 top 几行，并画一个最直观的：
- robustness（失败率）随 outlier_mult 的曲线（跨季聚合后，更适合写论文）

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

summary_fp = REPO_ROOT / 'outputs' / 'tables' / 'mcm2026c_q4_sensitivity_summary.csv'

if not summary_fp.exists():
    print('⚠️ Summary table not found yet:', summary_fp)
    print('Run Q4 once (one-click cell) to generate it.')
else:
    s = pd.read_csv(summary_fp)
    print('✅ Loaded summary:', s.shape)
    display(s.head(12))

    # 画一个最直观的：robust_fail_mean vs outlier_mult（跨季度聚合）
    if {'mechanism', 'outlier_mult', 'robust_fail_mean'}.issubset(set(s.columns)):
        fig, ax = plt.subplots(figsize=(10, 5))

        for mech, g in s.groupby('mechanism', sort=True):
            g = g.sort_values('outlier_mult')
            xs = pd.to_numeric(g['outlier_mult'], errors='coerce')
            ys = pd.to_numeric(g['robust_fail_mean'], errors='coerce')
            ax.plot(xs, ys, marker='o', linewidth=2, label=str(mech))

        ax.set_title('Mainline Q4 sensitivity summary: robustness vs outlier strength (across seasons)')
        ax.set_xlabel('outlier_mult')
        ax.set_ylabel('robust_fail_mean (lower is better)')
        ax.set_ylim(0, 1)
        ax.grid(True, alpha=0.3)
        ax.legend(bbox_to_anchor=(1.02, 1), loc='upper left')
        plt.tight_layout()
        plt.show()

    else:
        print('⚠️ Missing expected columns for plotting.')
        print('Columns:', s.columns.tolist())


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 只看一个 sigma_scale（默认 1.0）和 alpha（默认 0.5）
df = m.copy()
if 'sigma_scale' in df.columns:
    df = df[df['sigma_scale'].astype(float) == float(df['sigma_scale'].astype(float).dropna().iloc[0])].copy()

outlier0 = sorted(df['outlier_mult'].dropna().unique())[0]
d0 = df[df['outlier_mult'] == outlier0].copy()

# ---- 1) Tradeoff scatter（机制平均表现 + 误差条） ----
mechs = sorted(d0['mechanism'].astype(str).unique())
rows = []
for mech in mechs:
    g = d0[d0['mechanism'].astype(str) == mech].copy()
    if g.empty:
        continue

    x = float(pd.to_numeric(g['fan_vs_uniform_contrast'], errors='coerce').mean())
    y = float(pd.to_numeric(g['tpi_season_avg'], errors='coerce').mean())

    # fan 指标误差条：对每行的 SE 做 RMS，再乘 1.96（近似 95%）
    xse = pd.to_numeric(g.get('fan_vs_uniform_contrast_se', pd.Series([], dtype=float)), errors='coerce')
    xse = xse[np.isfinite(xse)]
    xerr = float(1.96 * np.sqrt(np.mean(np.square(xse)))) if len(xse) else 0.0

    # tpi 误差条：优先用 bootstrap CI 反推 SE；否则用 std/sqrt(n)
    if 'tpi_boot_p025' in g.columns and 'tpi_boot_p975' in g.columns:
        lo = pd.to_numeric(g['tpi_boot_p025'], errors='coerce')
        hi = pd.to_numeric(g['tpi_boot_p975'], errors='coerce')
        ok = np.isfinite(lo) & np.isfinite(hi)
        if ok.any():
            se = float(np.mean((hi[ok] - lo[ok]) / (2.0 * 1.96)))
            yerr = float(1.96 * se)
        else:
            yerr = 0.0
    else:
        std = pd.to_numeric(g.get('tpi_std', pd.Series([], dtype=float)), errors='coerce')
        n = pd.to_numeric(g.get('tpi_n', pd.Series([], dtype=float)), errors='coerce')
        ok = np.isfinite(std) & np.isfinite(n) & (n > 0)
        if ok.any():
            se = float(np.mean(std[ok] / np.sqrt(n[ok])))
            yerr = float(1.96 * se)
        else:
            yerr = 0.0

    rows.append({'mechanism': mech, 'fan_mean': x, 'fan_err': xerr, 'tpi_mean': y, 'tpi_err': yerr})

plot_df = pd.DataFrame(rows)

fig, ax = plt.subplots(figsize=(10, 6))
ax.scatter(plot_df['fan_mean'], plot_df['tpi_mean'], s=120, alpha=0.85)
ax.errorbar(plot_df['fan_mean'], plot_df['tpi_mean'],
            xerr=plot_df['fan_err'], yerr=plot_df['tpi_err'],
            fmt='none', ecolor='gray', alpha=0.4, capsize=3)

for _, r in plot_df.iterrows():
    ax.annotate(str(r['mechanism']).replace('_', '\n'), (r['fan_mean'], r['tpi_mean']),
                xytext=(6, 6), textcoords='offset points', fontsize=9)

ax.set_title(f'Tradeoff (outlier_mult={outlier0}): Fan expression vs TPI (with ~95% error bars)')
ax.set_xlabel('Fan expression (fan_vs_uniform_contrast)')
ax.set_ylabel('Technical Protection (tpi_season_avg)')
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.grid(True, alpha=0.3)
plt.show()

# ---- 2) Robustness curve（失败率 vs 压力强度 + 误差带） ----
fig, ax = plt.subplots(figsize=(10, 5))

outliers = sorted(df['outlier_mult'].dropna().unique())
for mech in mechs:
    g = df[df['mechanism'].astype(str) == mech].copy()
    if g.empty:
        continue

    xs, ys, bands = [], [], []
    for om in outliers:
        gg = g[g['outlier_mult'] == om].copy()
        if gg.empty:
            continue
        y = float(pd.to_numeric(gg['robust_fail_rate'], errors='coerce').mean())
        se = pd.to_numeric(gg.get('robust_fail_rate_se', pd.Series([], dtype=float)), errors='coerce')
        se = se[np.isfinite(se)]
        band = float(1.96 * np.sqrt(np.mean(np.square(se)))) if len(se) else float('nan')
        xs.append(float(om))
        ys.append(y)
        bands.append(band)

    if not xs:
        continue

    ax.plot(xs, ys, marker='o', linewidth=2, label=mech)
    if np.all(np.isfinite(bands)):
        lo = np.clip(np.array(ys) - np.array(bands), 0.0, 1.0)
        hi = np.clip(np.array(ys) + np.array(bands), 0.0, 1.0)
        ax.fill_between(xs, lo, hi, alpha=0.12)

ax.set_title('Robustness curve: fail rate under stronger outlier attack (with ~95% bands)')
ax.set_xlabel('outlier_mult (stress test strength)')
ax.set_ylabel('robust_fail_rate (lower is better)')
ax.set_ylim(0, 1)
ax.grid(True, alpha=0.3)
ax.legend(bbox_to_anchor=(1.02, 1), loc='upper left')
plt.tight_layout()
plt.show()

## 5) 生成 Q4 图（会自动使用误差列）

Q4 的可视化会：
- 如果 `robust_fail_rate_se` 存在，就用它画更合理的误差带
- 如果存在扩展攻击列且不是全 NaN，会自动把 `robust_fail_rate` 替换为多攻击下的 worst-case（更保守口径）

图输出：`outputs/figures/Q4/`（TIFF + EPS）。

In [ ]:
from mcm2026.visualizations.config import VisualizationConfig, create_output_directories
from mcm2026.visualizations.q4_visualizations import generate_all_q4_visualizations

viz_cfg = VisualizationConfig()
out_dirs = create_output_directories(REPO_ROOT / 'outputs' / 'figures', ['Q4'])
generate_all_q4_visualizations(REPO_ROOT, out_dirs['Q4'], viz_cfg, showcase=False)
out_dirs['Q4']
